In [1]:
from gurobipy import *  # import the optimize solver Gurobi

In [2]:
number_of_product = 3 # Set the index i for number of type product(Extracted lard oil, lean pork, dish)
number_of_j = 3 # Set the index j for two knid of pig(white big, black big) and dish
m = Model() # Import and create the model

Using license file C:\Users\chaoz\gurobi_licence\gurobi.lic
Academic license - for non-commercial use only


In [3]:
# Set the input Parameter: 
unit_produce = [[2, 1, 0], [3, 2, 0], [0, 0, 1]]  # Units of product i produced from one unit of j
unit_dish = [2, 3, 0] # Units of product i used to produce one unit of dish
process_cost = [6, 3, 5] # Processing cost of j
quality_level = [8, 7, 6] # Quality level of each unit of product i
avg_quality_level = 7  #  Average quality level on units sold
selling_price = [5, 4, 25] # Selling price of each unit of product i
max_unit = [2000, 2500, 800] # Maximum selling units of product i 

In [6]:
# Set the Variable list: :Units of j used in the process 
# Set the variable nx to integer number
nx = [] 
for j in range(number_of_j):
    nx.append(m.addVar(vtype=GRB.INTEGER, name='nx{}'.format(j + 1)))

In [7]:
# Caculate total products produced in step 1
tp = []
for i in range(number_of_product):
    tp.append(quicksum([unit_produce[i][j] * nx[j] for j in range(number_of_j)]))

In [8]:
# Caculate products used to produce dish
dish = []
for i in range(number_of_product):
    dish.append(unit_dish[i] * nx[2])

In [9]:
# Caculate units of product i sold
sold = []
for i in range(number_of_product):
    sold.append(tp[i] - dish[i])

In [11]:
# Set the Maximize Obijective: Total profit
m.setObjective(quicksum([sold[i]*selling_price[i] for i in range(number_of_product)]) - quicksum([nx[j]*process_cost[j] for j in range(number_of_j)]),  GRB.MAXIMIZE)

In [17]:
# Set Non Negative decision variable
c1 = []
for j in range(len(nx)):
    c1.append(m.addConstr(nx[j] >= 0))
    
# units of product i sold cannot exceed maximum selling units 
c2 = []
for i in range(number_of_product):
    c2.append(m.addConstr(sold[i] <= max_unit[i]))

# Quality level must be beyond the average quality-level 
c3 = m.addConstr(quicksum([sold[i]*quality_level[i] for i in range(number_of_product)]) >= quicksum([sold[i]*avg_quality_level for i in range(number_of_product)]))


In [18]:
# Run the optimize solver
m.optimize()

Gurobi Optimizer version 9.0.1 build v9.0.1rc0 (win64)
Optimize a model with 28 rows, 3 columns and 42 nonzeros
Model fingerprint: 0xaea7c2f7
Variable types: 0 continuous, 3 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+00]
  Objective range  [2e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [8e+02, 3e+03]
Found heuristic solution: objective 13328.000000
Presolve removed 26 rows and 0 columns
Presolve time: 0.06s
Presolved: 2 rows, 3 columns, 6 nonzeros
Variable types: 0 continuous, 3 integer (0 binary)

Root relaxation: objective 2.453300e+04, 1 iterations, 0.03 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 24533.0000    0    1 13328.0000 24533.0000  84.1%     -    0s
H    0     0                    24528.000000 24533.0000  0.02%     -    0s
H    0     0                    24532.000000 24533.0000  0.00%     -    0s


In [19]:
# Get the Optimal Solution for X
m.printAttr('X')


    Variable            X 
-------------------------
         nx1         1632 
         nx2            2 
         nx3          800 


In [20]:
# Get the Optimal Objective Value
m.ObjVal

24532.0

In [27]:
# Get the units sold

tp = []
for i in range(number_of_product):
    tp.append(quicksum([unit_produce[i][j] * m.x[j] for j in range(number_of_j)]))

dish = []
for i in range(number_of_product):
    dish.append(unit_dish[i] * m.x[2])

sold = []
for i in range(number_of_product):
    sold.append(tp[i] - dish[i])
    print('Uints sold:', sold[i])

Uints sold: <gurobi.LinExpr: 1666.0>
Uints sold: <gurobi.LinExpr: 2500.0>
Uints sold: <gurobi.LinExpr: 800.0>
